In [96]:
import numpy as np
# seed = 123
# np.random.seed(seed)
import collections, copy, pickle
from importlib import reload
from dateutil.parser import parse
import scipy.linalg, scipy.stats
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
from matplotlib import rcParams
rcParams['font.family'] = 'serif'
rcParams['font.size'] = 14
# rcParams['text.usetex'] = True
import seaborn as sns
from IPython.display import HTML
import scipy.stats
from sklearn import preprocessing
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [97]:
import util.data
import util.ndcg
# import util.plot

In [98]:
def sort_pred_test(x_test, y_test, y_pred):
    # calculate dcg of test set per srch_id
    Xy_pred = util.data.Xy_pred(x_test, y_pred)
    # put true y values on indexes, do not sort !
    Xy_true = util.data.Xy_pred(x_test, y_test)
    return Xy_pred, Xy_true

def dcg_at_k(r, k, method=0):
    r = np.asfarray(r)[:k]
    if r.size:
        if method == 0:
            return r[0] + np.sum(r[1:] / np.log2(np.arange(2, r.size + 1)))
        elif method == 1:
            return np.sum(r / np.log2(np.arange(2, r.size + 2)))
        else:
            raise ValueError('method must be 0 or 1.')
    return 0.

def ndcg_at_k(r, k, method=0):
    dcg_max = dcg_at_k(sorted(r, reverse=True), k, method)
    if not dcg_max:
        return 0.
    return dcg_at_k(r, k, method) / dcg_max

def DCG_dict(data):
    DCG = {}
#     for id in data['srch_id']:
    # rows = rows_srch_id(data, id)
    # r = relevance_scores(rows)
    r = []
    prev_srch_id = -1
    position = 0
    for i in data.index.tolist():
        if prev_srch_id == -1:
            row = data.loc[i]
            cur_srch_id = row.srch_id
            prev_srch_id = 0
        row = data.loc[i]
        next_id = row.srch_id
        score = row.score
        # compute position
        if cur_srch_id != next_id:
#             dcg = dcg_at_k(r, len(r), method=0)
            DCG[cur_srch_id] = ndcg_at_k(r, k=len(r))
            cur_srch_id = next_id
            r = []
            r.append(score)
            position += 1
        else:
            r.append(score)
            position += 1
#     dcg = dcg_at_k(r, len(r), method=0)
    DCG[cur_srch_id] = ndcg_at_k(r, k=len(r))
    return DCG

In [111]:
data = pd.read_csv('data/training_set_VU_DM_clean.csv', sep=';', nrows=100000)
data_test = pd.read_csv('data/test_set_VU_DM.csv', sep=',', nrows=100000)

In [112]:
srchIdList = data['srch_id'].unique()
trainID, testID = sklearn.model_selection.train_test_split(srchIdList, random_state=42, test_size=0.4)
trainset = data[data['srch_id'].isin(trainID)]
testset = data[data['srch_id'].isin(testID)]

y_labels = ['click_bool', 'booking_bool', 'score']

def split_xy(data: pd.DataFrame):
    return data.drop(columns=y_labels), data['score']

X_train, y_train = split_xy(trainset)
# X_val, y_val = split_xy(valset)
X_test, y_test = split_xy(testset)

X_train = X_train.drop('position', 1)
X_test = X_test.drop('position', 1)

In [113]:
# use Gradient Boosting to make model of data to predict the 'score'
xg_reg = xgb.XGBRegressor(objective ='rank:ndcg', colsample_bytree = 0.3, learning_rate = 0.1, 
                          max_depth = 5, alpha = 10, n_estimators = 10)

# fit training set
xg_reg.fit(X_train,y_train)

# fit model to test set
y_pred = xg_reg.predict(X_test)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


In [ ]:
Xy_pred = X_test.copy([['srch_id','prop_id','score']])
Xy_true = X_test.copy([['srch_id','prop_id','score', 'position']])
Xy_pred['score'] = y_pred
Xy_true['score'] = y_test
Xy_pred.sort_values(['srch_id', 'score'], ascending=[True, False])
Xy_true.sort_values(['srch_id', 'score'], ascending=[True, False])
Xy_pred['score'] = y_test
# Xy_pred, Xy_true = sort_pred_test(X_test, y_test, y_pred)
dcg_test = DCG_dict(Xy_pred)
dcg_control = DCG_dict(Xy_true)

# ndcg = np.mean(np.array(list(dcg_test.values())) / np.array(list(dcg_control.values())))
# print(ndcg)
ndcg = np.mean(np.array(list(dcg_test.values())))
ndcg

In [115]:
# # calculate dcg of test set per srch_id
# Xy_pred = util.data.Xy_pred(X_test, y_pred)

# # put true y values on indexes, do not sort !
# Xy_pred['score'] = y_test

# # calculate ideal dcg of test set per srch_id
# Xy_true = util.data.Xy_pred(X_test, y_test, y_pred)

# calculate NDCG
# ndcg = util.ndcg.ndcg(X_test, y_test, y_pred)
# ndcg

In [103]:
dcg_test

{1.0: 0.27023815442731974,
 17.0: 0.22424382421757544,
 21.0: 0.24465054211822604,
 25.0: 0.4306765580733931,
 31.0: 1.0,
 39.0: 0.24465054211822604,
 43.0: 0.21810429198553155,
 49.0: 0.38685280723454163,
 52.0: 0.2890648263178878,
 56.0: 0.5,
 57.0: 0.2103099178571525,
 63.0: 0.3562071871080222,
 64.0: 0.21810429198553158,
 65.0: 0.23981246656813146,
 66.0: 0.3010299956639812,
 67.0: 0.6309297535714574,
 80.0: 0.20801459767650946,
 82.0: 0.38685280723454163,
 83.0: 0.21533827903669653,
 84.0: 1.0,
 90.0: 0.21274605355336318,
 96.0: 0.21274605355336318,
 97.0: 0.4306765580733931,
 103.0: 0.5,
 106.0: 0.40571676760716774,
 107.0: 0.26264953503719357,
 109.0: 0.43067655807339306,
 111.0: 0.23981246656813146,
 113.0: 0.2242438242175754,
 115.0: 0.6309297535714574,
 117.0: 0.38685280723454163,
 118.0: 0.2242438242175754,
 123.0: 0.21533827903669656,
 127.0: 1.0,
 129.0: 0.22106472945750374,
 133.0: 0.27023815442731974,
 140.0: 1.0,
 143.0: 0.5,
 144.0: 1.0,
 147.0: 0.3562071871080222,
 15

In [104]:
dcg_control

{1.0: 0.27023815442731974,
 17.0: 0.22424382421757544,
 21.0: 0.24465054211822604,
 25.0: 0.4306765580733931,
 31.0: 1.0,
 39.0: 0.24465054211822604,
 43.0: 0.21810429198553155,
 49.0: 0.38685280723454163,
 52.0: 0.2890648263178878,
 56.0: 0.5,
 57.0: 0.2103099178571525,
 63.0: 0.3562071871080222,
 64.0: 0.21810429198553158,
 65.0: 0.23981246656813146,
 66.0: 0.3010299956639812,
 67.0: 0.6309297535714574,
 80.0: 0.20801459767650946,
 82.0: 0.38685280723454163,
 83.0: 0.21533827903669653,
 84.0: 1.0,
 90.0: 0.21274605355336318,
 96.0: 0.21274605355336318,
 97.0: 0.4306765580733931,
 103.0: 0.5,
 106.0: 0.40571676760716774,
 107.0: 0.26264953503719357,
 109.0: 0.43067655807339306,
 111.0: 0.23981246656813146,
 113.0: 0.2242438242175754,
 115.0: 0.6309297535714574,
 117.0: 0.38685280723454163,
 118.0: 0.2242438242175754,
 123.0: 0.21533827903669656,
 127.0: 1.0,
 129.0: 0.22106472945750374,
 133.0: 0.27023815442731974,
 140.0: 1.0,
 143.0: 0.5,
 144.0: 1.0,
 147.0: 0.3562071871080222,
 15